## Подготовка данных

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

*Пояснение действий:*
1. Загрузил все необходимые библиотеки

In [2]:
df = pd.read_csv('/content/sample_data/Churn.csv')

In [3]:
display(df)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Пояснения данных:
RowNumber — индекс строки в данных

CustomerId — уникальный идентификатор клиента

Surname — фамилия

CreditScore — кредитный рейтинг

Geography — страна проживания

Gender — пол

Age — возраст

Tenure — количество недвижимости у клиента

Balance — баланс на счёте

NumOfProducts — количество продуктов банка, используемых клиентом

HasCrCard — наличие кредитной карты (1 - есть, 0 - нет)

IsActiveMember — активность клиента (1 - есть, 0 - нет)

EstimatedSalary — предполагаемая зарплата

Exited — факт ухода клиента (1 - да, 0 - нет)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
df.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

*Пояснение действий:*
1. Исследовал данные, в столбце Tenure — количество недвижимости у клиента, обнаружил 909 пропусков. Скорее всего в этом случае недвижимости просто нет, т.к. в банковском деле заполнение этой графы немало важно, поэтому принимаю решение заполнить 0.

In [6]:
df['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

In [7]:
(df['Tenure']==0).value_counts()

False    9618
True      382
Name: Tenure, dtype: int64

In [8]:
df['Tenure'] = df['Tenure'].fillna(0)
df['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0.])

In [9]:
df['Tenure'].value_counts()

0.0     1291
1.0      952
2.0      950
8.0      933
3.0      928
5.0      927
7.0      925
4.0      885
9.0      882
6.0      881
10.0     446
Name: Tenure, dtype: int64

In [10]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

*Пояснение действий:*
1. Заполнил и проверил, остальные данные без замечаний.
2. Обнаружил, что в таблице есть ненужные нам данные RowNumber, CustomerId и Surname. Они не пригодятся при исследовании модели, поэтому удалю их.
3. Приведу названия столюцов к нижнему регистру.

In [11]:
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [13]:
df.columns.tolist()

['CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [14]:
df.rename(columns = {'CreditScore':'Место в рейтинге',
'Geography':'geography',
'Gender':'gender',
'Age':'age',
'Tenure':'tenure',
'Balance':'balance',
'NumOfProducts':'numofproducts',
'HasCrCard':'hascrcard',
'IsActiveMember':'isactivemember',
'EstimatedSalary':'estimatedsalary',
'Exited':'exited'},
inplace = True)

df.head()

,Место в рейтинге,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


**Вывод:**
Данные подготовлены к работе.

## Исследование задачи

Научну с защиты от дамми ловушек

In [15]:
df = pd.get_dummies(df, drop_first=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Место в рейтинге   10000 non-null  int64  
 1   age                10000 non-null  int64  
 2   tenure             10000 non-null  float64
 3   balance            10000 non-null  float64
 4   numofproducts      10000 non-null  int64  
 5   hascrcard          10000 non-null  int64  
 6   isactivemember     10000 non-null  int64  
 7   estimatedsalary    10000 non-null  float64
 8   exited             10000 non-null  int64  
 9   geography_Germany  10000 non-null  uint8  
 10  geography_Spain    10000 non-null  uint8  
 11  gender_Male        10000 non-null  uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


По опыту предудущего проекта проведу разобиение данных по принципу 60/20/20.

In [16]:
target = df['exited']
features = df.drop('exited', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4,
                                                                              random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5,
                                                                              random_state=12345 )

scaler = StandardScaler()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train, copy=True)
features_valid_scaled = scaler.transform(features_valid, copy=True)


print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)
print(features_train_scaled.shape,features_valid_scaled.shape)

features_train = pd.DataFrame(features_train_scaled)
features_valid = pd.DataFrame(features_valid_scaled)

(6000, 11) (2000, 11) (2000, 11)
(6000,) (2000,) (2000,)
(6000, 11) (2000, 11)


In [17]:
target_train.reset_index(drop=True,inplace=True)
features_test = scaler.transform(features_test)

In [18]:
target.value_counts()

0    7963
1    2037
Name: exited, dtype: int64

Присутствует дисбаланс классов. Посмотрим как покажут себя исследования.

In [21]:
%%time

model =  LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('F1', f1_score(target_valid, predicted_valid))

F1 0.3333333333333333
CPU times: user 20.5 ms, sys: 7.83 ms, total: 28.3 ms
Wall time: 23.3 ms


*Пояснение действий:*

1. Провел исследование методом логистическая регрессия.
2. Результат очень низкий из за низкого precision или recall скорее всего. Придется смотреть их.


In [22]:
%%time

model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Логистическая регрессия')
print('Точность попадания по классам', accuracy_score(target_valid, predicted_valid))
print('precision', precision_score(target_valid, predicted_valid))
print('recall', recall_score(target_valid, predicted_valid))
print('F1', f1_score(target_valid, predicted_valid))
print()

Логистическая регрессия
Точность попадания по классам 0.802
precision 0.5625
recall 0.23684210526315788
F1 0.3333333333333333

CPU times: user 43.3 ms, sys: 14.9 ms, total: 58.2 ms
Wall time: 89.3 ms


In [23]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC Логистическая регрессия',auc_roc)

AUC-ROC Логистическая регрессия 0.7585652586817002


## Борьба с дисбалансом

Для борьбы с дисбалансом классов необходимо изучить модель по взвешиванию классов и исследовать способы изучнения классов.

In [24]:
%%time
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Логистическая регрессия')
print('Точность попадания по классам', accuracy_score(target_valid, predicted_valid))
print('pecision', precision_score(target_valid, predicted_valid))
print('recall', recall_score(target_valid, predicted_valid))
print('F1', f1_score(target_valid, predicted_valid))
print()

Логистическая регрессия
Точность попадания по классам 0.701
pecision 0.3803191489361702
recall 0.6842105263157895
F1 0.4888888888888888

CPU times: user 35.8 ms, sys: 13 ms, total: 48.7 ms
Wall time: 32.4 ms


*Пояснение действий:*

1. Изучил модель по взвешиванию классов.
2. F1 сильно возросла в Логистической регрессии.
3. Теперь предстоит работа с дисбалансом. Для этого необходимо увеличить значения класса 1 и уменьшить значения класса 0.

In [25]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

print(target_upsampled.value_counts())

0    4804
1    4784
Name: exited, dtype: int64


In [26]:
features_train.head()

,0,1,2,3,4,5,6,7,8,9,10
0,-0.886751,-0.373192,1.104696,1.232271,-0.891560,0.642466,-1.055187,-0.187705,-0.572475,1.728977,0.907278
1,0.608663,-0.183385,1.104696,0.600563,-0.891560,-1.556504,-1.055187,-0.333945,-0.572475,-0.578377,-1.102198
2,2.052152,0.480939,-0.503694,1.027098,0.830152,-1.556504,0.947699,1.503095,1.746802,-0.578377,0.907278
3,-1.457915,-1.417129,0.461340,-1.233163,0.830152,0.642466,-1.055187,-1.071061,-0.572475,-0.578377,0.907278
4,0.130961,-1.132419,-0.825373,1.140475,-0.891560,-1.556504,-1.055187,1.524268,1.746802,-0.578377,-1.102198


In [27]:
target_train.head()

0    0
1    0
2    0
3    0
4    0
Name: exited, dtype: int64

In [28]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

print(target_downsampled.value_counts())

0    1201
1    1196
Name: exited, dtype: int64


In [29]:
%%time
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('Логистическая регрессия')
print('Точность попадания по классам', accuracy_score(target_valid, predicted_valid))
print('precision', precision_score(target_valid, predicted_valid))
print('recall', recall_score(target_valid, predicted_valid))
print('F1', f1_score(target_valid, predicted_valid))
print()

Логистическая регрессия
Точность попадания по классам 0.802
precision 0.5625
recall 0.23684210526315788
F1 0.3333333333333333

CPU times: user 36.7 ms, sys: 14 ms, total: 50.7 ms
Wall time: 115 ms


**Выводы**
1. Выполнил борьбу с дисбалансом